In [ ]:
from json import load
import os 
from pyprojroot import here
import pandas as pd
import chromadb
from openai import AzureOpenAI
import warnings
warnings.filterwarnings("ignore")
from dotenv import load_dotenv
print(load_dotenv())

In [ ]:
azure_openai_api_key = os.environ["OPENAI_API_KEY"]
azure_openai_endpoint = os.environ["OPENAI_API_BASE"]

In [ ]:
azure_client = AzureOpenAI(
    api_key = azure_openai_api_key,
    api_version = "2024-06-30",
    azure_endpoint = azure_openai_endpoint,
)
chroma_client = chromadb.PersistentClient(path=str(here("data/chroma_db")))

In [ ]:
collection = chroma_client.create_collection(name="csv_sql_db_collection")


In [ ]:
file_dir = here("data/csv_sql_db/data/api_data.csv")
df = pd.read_csv(file_dir, rows=10)


In [ ]:
df

In [ ]:
docs = []
metadata = []
ids = []
embeddings = []
for index, row in df.iterrows():
    output_str = ""
    
    for col in df.columns:
        output_str += f"{col}: {row[col]}\n"
    response = azure_client.embeddings.create(
        input=output_str,
        model="text-embedding-ada-002"
    )
    embeddings.append(response.data[0].embedding)
    docs.append(output_str)
    metadata.append({"source": "api_data.csv"})
    ids.append(f"str{index}")

In [ ]:
docs

In [ ]:
print(metadatas)
print(ids)

In [ ]:
embeddings[0][:10]

In [ ]:
collection.add(
    documents=docs,
    metadatas=metadata,
    ids=ids,
    embeddings=embeddings
)

In [ ]:
print("Number of documents in the collection:", collection.count())

In [ ]:
from openai import AzureOpenAI

In [ ]:
model_name = "gpt-35-turbo"
azure_openai_api_key = os.environ["OPENAI_API_KEY"]
azure_openai_endpoint = os.environ["OPENAI_API_BASE"]

In [ ]:
azure_client = AzureOpenAI(
    api_version = os.getenv("OPENAI_API_VERSION"),
    azure_endpoint = os.getenv("OPENAI_API_BASE"),
    api_key = azure_openai_api_key,
)

In [ ]:
query_texts = "I need more info about Own Harris"
response = azure_client.chat.completions.create(
    input = query_texts,
    model = "text-embedding-ada-002"
)
query_embeddings = response.data[0].embedding



In [ ]:
vectordb = chroma_client.get_collection(name="csv_sql_db_collection")
vectordb.count()


In [ ]:
results = vectordb.query(
    query_embeddings = query_embeddings, n_results=1
)

results

In [ ]:
system_role = """You are a helpful AI assistant. Use the provided context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer."""
prompt = f"User's question: {query_texts} \n\n Search results:\n {results}"

messages = [
    {"role": "system", "content": str(
        system_role
        )},
    {"role": "user", "content": prompt}
]

In [ ]:
response = azure_client.chat.completions.create(
    model=os.getenv("gpt_deloyment_name"),
    messages=messages,
)

In [ ]:
response.choices[0].message.content

In [ ]:
df